In [12]:
 import rasterio

# Open a tiff file

In [13]:
d_alt = rasterio.open(r'C:\Users\emack\Documents\UAF-Temperature\data\alt.tif')

# Get the band count

In [14]:
d_alt.count

1

###  Get the dimensions of the band arrays

In [15]:
print('Band width is ',d_alt.width)
print('Band height is ',d_alt.height)

Band width is  43200
Band height is  17882


### Get the mapping of band indexes to variable type

In [16]:
{i: dtype for i, dtype in zip(d_alt.indexes, d_alt.dtypes)}

{1: 'int16'}

# Dataset georeferencing
Get the bounds of our dataset

In [17]:
d_alt.bounds

BoundingBox(left=-180.0, bottom=-60.016666667535816, right=180.00001877546288, top=89.00000777095556)

### A dataset’s transform is an affine transformation matrix that maps pixel locations in (row, col) coordinates to (x, y) spatial positions. The product of this matrix and (0, 0), the row and column coordinates of the upper left corner of the dataset, is the spatial position of the upper left corner.

In [18]:
d_alt.transform

Affine(0.00833333376795053, 0.0, -180.0,
       0.0, -0.00833333376795053, 89.00000777095556)

### Get the position of the lower right corner

In [19]:
d_alt.transform * (0, 0)

(-180.0, 89.00000777095556)

# Get the coordinate reference system (CRS)

In [20]:
d_alt.crs

CRS.from_dict(init='epsg:4326')

# Spatial indexing

Datasets have an index() method for getting the array indices corresponding to points in georeferenced space. To get the value for the pixel 100 kilometers east and 50 kilometers south of the dataset’s upper left corner, do the following.

In [29]:
d_alt.indexes
band1 = d_alt.read(1)

In [30]:
band1

array([[-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       ...,
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768]], dtype=int16)

### To get the spatial coordinates of a pixel, use the dataset’s xy() method. The coordinates of the center of the image can be computed like this.

In [57]:
band1[dataset.height // 2, dataset.width // 2]

291

In [55]:
x, y = (d_alt.bounds.left -180, d_alt.bounds.top + 89)
row, col = d_alt.index(x, y)
row, col

band1[row, col]


178